In [16]:
import random
import time
import numpy as np
import torch

In [17]:
def generate_sequence(len_seq):
    x = np.random.randint(0, 9, len_seq)
    y = []
    y.append(x[0])
    for i in x[1:]:
        yi = i + x[0]
        if yi >= 10:
            yi = yi - 10
        y.append(yi)
    y = np.array(y)
    return x , y

In [18]:
def scaling_window(X, Y, seq_length):
    x = []
    y = []

    for i in range(len(X)-seq_length-1):
        _x = X[i:(i+seq_length)]
        _y = Y[i:(i+seq_length)]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)


x, y = generate_sequence(10000)

seq_length = 6
xw, yw = scaling_window(x, y, seq_length)

In [19]:
class RnnFlex(torch.nn.Module):

    def __init__(self, rnnClass):
        super().__init__()
        self.embed = torch.nn.Embedding(10, 32)
        self.hidden = rnnClass(32, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 10)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, h = self.hidden(embed)
        return self.linear(o)

In [20]:
X = torch.zeros((len(xw), seq_length), dtype=int)
Y = torch.zeros((len(xw), seq_length), dtype=int)

for i, seq in enumerate(xw):
    for t, num in enumerate(seq):
        X[i, t] = num
        Y[i, t] = yw[i, t]

In [21]:
dataset = torch.utils.data.TensorDataset(X, Y)

BATCH_SIZE = 32

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dl = torch.utils.data.DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, BATCH_SIZE, shuffle=True)

In [22]:

num_epochs = 20
learning_rate = 0.01

model = RnnFlex(torch.nn.RNN)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    for x_in, y_in in train_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        optimizer.zero_grad()
        out = model.forward(x_in).view(-1, 10)
        l = criterion(out, y_in)
        train_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]
        l.backward()
        optimizer.step()
        iter_num += 1
    print(
        f"Epoch: {epoch}, loss: {train_loss:.4f}, acc: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )
    test_loss, test_acc, iter_num = .0, .0, .0
    model.eval()
    for x_in, y_in in test_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        out = model.forward(x_in).view(-1, 10)
        l = criterion(out, y_in)
        test_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        test_acc += batch_acc.sum().item() / batch_acc.shape[0]
        iter_num += 1
    print(
        f"test loss: {test_loss:.4f}, test acc: {test_acc / iter_num:.4f} | "
        f"{time.time() - start_epoch_time:.2f} sec."
    )

Epoch: 0, loss: 6.5250, acc: 0.9958 | test loss: 0.0914, test acc: 0.9999 | 1.12 sec.
Epoch: 1, loss: 0.0043, acc: 1.0000 | test loss: 0.0918, test acc: 0.9999 | 1.26 sec.
Epoch: 2, loss: 0.0026, acc: 1.0000 | test loss: 0.0924, test acc: 0.9999 | 1.11 sec.
Epoch: 3, loss: 0.0018, acc: 1.0000 | test loss: 0.0931, test acc: 0.9999 | 1.10 sec.
Epoch: 4, loss: 0.0013, acc: 1.0000 | test loss: 0.0938, test acc: 0.9999 | 1.06 sec.
Epoch: 5, loss: 0.0010, acc: 1.0000 | test loss: 0.0945, test acc: 0.9999 | 1.25 sec.
Epoch: 6, loss: 0.0008, acc: 1.0000 | test loss: 0.0951, test acc: 0.9999 | 1.01 sec.
Epoch: 7, loss: 0.0006, acc: 1.0000 | test loss: 0.0958, test acc: 0.9999 | 1.20 sec.
Epoch: 8, loss: 0.0005, acc: 1.0000 | test loss: 0.0964, test acc: 0.9999 | 1.20 sec.
Epoch: 9, loss: 0.0004, acc: 1.0000 | test loss: 0.0970, test acc: 0.9999 | 1.06 sec.
Epoch: 10, loss: 0.0004, acc: 1.0000 | test loss: 0.0975, test acc: 0.9999 | 1.13 sec.
Epoch: 11, loss: 0.0003, acc: 1.0000 | test loss: 0.0

In [23]:
def predict():
    x_in, y_in = random.choice(test_dataset)
    model.eval()
    outputs = model.forward(x_in).view(-1, 10)
    print('x: {},\ny: {},\ny_pred: {}'.format(x_in, y_in, outputs.argmax(dim=1)))

    val_acc = (outputs.argmax(dim=1) == y_in).flatten()
    val_acc = (val_acc.sum() / val_acc.shape[0]).item()
    print('Accuracy = {:.2f}'.format(val_acc))

predict()

x: tensor([4, 3, 8, 7, 2, 8]),
y: tensor([0, 9, 4, 3, 8, 4]),
y_pred: tensor([0, 9, 4, 3, 8, 4])
Accuracy = 1.00
